In [5]:
# Lab1 : Threshold Secret Sharing
from book.vecutil import list2vec
from book.GF2 import one
from book.vec import Vec

In [3]:
a0 = list2vec([one, one, 0, one, 0, one])
b0 = list2vec([one, one, 0, 0, 0, one])

In [4]:
import random
def randGF2():
    return random.randint(0, 1)*one


In [14]:
# Task 7.7.1
# Two methods:
# - Brute force, picking until the condition is met
# - Pick and modify
def choose_secret_vector(s, t):
    "Version 1: brute force"
    D = set(range(6))
    while True:
        u = Vec(D, {i: randGF2() for i in D})
        if (a0 * u == s and b0 * u == t):
            return u

s = one
t = 0
u = choose_secret_vector(s, t)
[
    u*a0,
    u*b0
]

[one, 0]

In [417]:
from book.independence import is_independent
import itertools

def meet_requirement(all_vectors):
    # For any three pairs, the corresponding six vectors are linearly independent.
    if all([is_independent([all_vectors[vi] for vi in flatten(pairs)])
        for pairs in itertools.combinations([(0, 1), (2, 3), (4, 5), (6, 7), (8, 9)], 3)]):
            return True
    return False

def choose_random_vectors(a0, b0):
    """
    We can choose the vectors randomly until we find a set that satisfies the requirement. Here's one way to do it:
    """
    D = a0.D
    n = 0  # NOTE: to monitor the time expenses
    known = [a0, b0] 

    def flatten(pairs):
        return functools.reduce(
            lambda e, acc: acc + e, [[pair[0], pair[1]] for pair in pairs])

    while True:
        n += 1
        a1 = Vec(D, {i: randGF2() for i in D})
        b1 = Vec(D, {i: randGF2() for i in D})
        a2 = Vec(D, {i: randGF2() for i in D})
        b2 = Vec(D, {i: randGF2() for i in D})
        a3 = Vec(D, {i: randGF2() for i in D})
        b3 = Vec(D, {i: randGF2() for i in D})
        a4 = Vec(D, {i: randGF2() for i in D})
        b4 = Vec(D, {i: randGF2() for i in D})
        choices = [a1, b1, a2, b2, a3, b3, a4, b4]
        all_vectors = known + choices

        # ensure sets are unique and doesn't include a0, b0
        if len(set(all_vectors)) != len(all_vectors):
            continue

        if meet_requirement(all_vectors):
            return choices

[a1, b1, a2, b2, a3, b3, a4, b4] = choose_random_vectors(a0, b0)
all_vectors = [a0, b0] + [a1, b1, a2, b2, a3, b3, a4, b4]
print(meet_requirement(all_vectors))

True


In [418]:
from book.bitutil import *

message = "Euclid's algorithm for greatest common divisor"
# message = "Eu"
def encode(m):
    bitlist = str2bits(m)
    return bits2mat(bitlist, 2)

def decode(m):
    bitlist = mat2bits(m)
    return bits2str(bitlist)

decode(encode(message))

"Euclid's algorithm for greatest common divisor"

In [419]:
from book.matutil import *

def shape(M):
    return (len(M.D[0]), len(M.D[1]))

# message in 2xn matrix format
M = encode(message)
n = shape(M)[1]
# construct the secret vector, [a0, b0]^T x U would recover the message
U = coldict2mat({i: choose_secret_vector(col[0], col[1]) for i, col in mat2coldict(M).items()})
assert rowdict2mat([a0, b0]) * U == M

# Distribute keys
keys = rowdict2mat(all_vectors) * U
assert shape(keys) == (10, n)
keys = [row for _, row in mat2rowdict(keys).items()]

key_pairs = list(zip(keys, all_vectors))[2:] # ignore the pair for a0 and b0


In [421]:
from solve import solve
from book.independence import rank
import functools

# Recover U with six pairs
def combined_solve(pairs):
    """
    Solve each N column in U; each solve is using Gaussian Elimination for M = 8
    """
    assert len(pairs) >= 6

    # unzip pairs into two lists
    key_list, a_b_list = zip(*pairs)

    # Check that the matrix formed by the six pairs has full rank
    if rank(list(a_b_list)) < 6:
        raise Exception("a_b_list is not independent")

    AB = rowdict2mat(a_b_list)
    KM = rowdict2mat(key_list)

    U = coldict2mat({i: solve(AB, col) for i, col in mat2coldict(KM).items()})

    assert shape(AB)[1] == 6
    assert shape(KM)[1] == n

    return U

for index_pairs in itertools.combinations([(0, 1), (2, 3), (4, 5), (6, 7)], 3):
    six_pairs = functools.reduce(
        lambda e, acc: acc + e, [[key_pairs[pair[0]], key_pairs[pair[1]]] for pair in index_pairs])

    solved_U = combined_solve(six_pairs)
    solved_M = rowdict2mat([a0, b0]) * solved_U 
    if solved_M == M:
        print(decode(solved_M))
    else:
        print("!", [key_pairs.index(p) for p in six_pairs])

Euclid's algorithm for greatest common divisor
Euclid's algorithm for greatest common divisor
Euclid's algorithm for greatest common divisor
Euclid's algorithm for greatest common divisor


In [431]:
# Lab2 : Factoring integers
from book.factoring_support import *

def root_method(N):
    a = intsqrt(N) + 1 # Initialize a to be an integer greater than √N
    while True:
        b = a ** 2 - N
        sqrt_b = intsqrt(b)
        if sqrt_b ** 2 == b:
            return a - sqrt_b
        else:
            a += 1

[
    root_method(55),
    root_method(77),
    root_method(146771),
    # root_method(118),  # run forever, does it terminate?
]

[5, 1, 317]

In [435]:
from random import randint
r = randint(10, 34821578)
s = randint(30, 9848123405)
t = randint(10, 7417587061789234)

a = r * s
b = s * t

[
    r, s, t,
    gcd(a, b)
]

[27505838, 9187027503, 3663646682816225, 9187027503]

In [437]:
# (a-b)(a+b) = kN, there is chance (a-b) and N share primes

N = 367160330145890434494322103
a = 67469780066325164
b = 9429601150488992

gcd(N, a - b)

19117318483477

In [561]:
# Try to create a, b, first rewrite an integer in primeset coefficient format
def dumb_factor(x, primeset):
    if not primeset:
        return []

    primes = list(primeset)
    first, rest = primes[0], primes[1:]

    n = 0
    while x % (first ** n) == 0:
        n += 1
    assert x % (first ** n) != 0
    assert x % (first ** (n - 1)) == 0

    factor = (first, (n - 1))
    if n - 1 == 0:
        return dumb_factor(x, rest)

    remainder = x / (first ** (n - 1))

    if remainder == 1:
        return [factor]
    
    rest_factors = dumb_factor(remainder, rest)
    if rest_factors:
        return [factor] + rest_factors
    else:
        return []
    
[
    dumb_factor(75, {2, 3, 5, 7}),
    dumb_factor(30, {2, 3, 5, 7}),
    dumb_factor(1176, {2,3,5,7}),
    dumb_factor(2*17, {2,3,5,7}),
    dumb_factor(2*3*5*19, {2,3,5,7})
]

# NOTE: Use the iterative version for performance
from book.factoring_support import dumb_factor

In [563]:
primeset={2, 3, 5, 7, 11, 13}

[dumb_factor(x, primeset) for x in 
[12,154,2*3*3*3*11*11*13,2*17,2*3*5*7*19]]

[[(2, 2), (3, 1)],
 [(2, 1), (7, 1), (11, 1)],
 [(2, 1), (3, 3), (11, 2), (13, 1)],
 [],
 []]

In [564]:
from book.GF2 import one, zero
def int2GF2(i):
    return one if i % 2 == 1 else zero

In [565]:
from book.vec import Vec


def make_Vec(primeset, factors):
    return Vec(primeset, {p: int2GF2(v) for p, v in dict(factors).items()})

[
    make_Vec({2, 3, 5, 7, 11}, [(3, 1)]),
    make_Vec({2, 3, 5, 7, 11}, [(2, 17), (3, 0), (5, 1), (11, 3)]),
]

[Vec({2, 3, 5, 7, 11},{3: one}),
 Vec({2, 3, 5, 7, 11},{2: one, 3: 0, 5: one, 11: one})]

In [579]:
def find_candidates(N, primeset):
    count = len(primeset) + 1

    roots = []
    rowlist = []
    x = intsqrt(N) + 2
    while len(roots) < count:
        factors = dumb_factor(x**2 - N, primeset)
        if factors:
            roots.append(x)
            rowlist.append(make_Vec(primeset, factors))
        x += 1
    return roots, rowlist

N = 2419
roots, rowlist = find_candidates(N, primes(32))
print(roots)

[51, 52, 53, 58, 61, 62, 63, 67, 68, 71, 77, 79]


In [581]:
import functools
N = 2419

def guess_factor(N, primeset, *numbers):
    x_to_factor = lambda x, N: dumb_factor(x**2 - N, primeset)
    factor_to_num = lambda f: reduce(lambda acc, x: acc * x[0] ** x[1], f, 1)

    a = prod(numbers)
    b = intsqrt(
        prod([factor_to_num(x_to_factor(n, N)) for n in numbers]))
    return gcd(a - b, N)

[
    guess_factor(N, primes(32), 53, 77),
    guess_factor(N, primes(32), 52, 67, 71)
]

[41, 2419]

In [583]:
from book.echelon import transformation_rows

def row_to_x(row, xs):
    return [xs[d] for d in row.D if row[d] == one]
    

M = transformation_rows(rowlist)

[
    prod([factor_to_num(x_to_factor(n, N)) for n in row_to_x(last_row, xs)]),
    guess_factor(N, primes(32), *row_to_x(M[-1], roots)),
    guess_factor(N, primes(32), *row_to_x(M[-2], roots))
]


[605361802500, 1, 41]

In [584]:
def find_a_and_b(v, roots, N):
    numbers = row_to_x(v, roots)
    primeset = v.D

    x_to_factor = lambda x, N: dumb_factor(x**2 - N, primeset)
    factor_to_num = lambda f: reduce(lambda acc, x: acc * x[0] ** x[1], f, 1)

    a = prod(numbers)
    b = intsqrt(
        prod([factor_to_num(x_to_factor(n, N)) for n in numbers]))
    return a, b

In [591]:
# Putting things together
def solve_prime(N, factor_limit):
    primeset = primes(factor_limit)
    roots, rowlist = find_candidates(N, primeset)
    # print(f"found potential roots: {len(roots)} {roots}")

    M = transformation_rows(rowlist, sorted(primeset, reverse=True))

    for row_index in range(len(roots) - 1, 0, -1):
        result = guess_factor(N, primeset, *row_to_x(M[row_index], roots))
        if result != 1 and result != N:
            return result, N/result
        row_index += 1
    return None

N = 2461799993978700679
# solve_prime(N, 10000)

found potential roots: 1230 [1569012433, 1569012450, 1569012452, 1569012468, 1569012473, 1569012474, 1569012498, 1569012503, 1569012516, 1569012518, 1569012521, 1569012537, 1569012560, 1569012562, 1569012569, 1569012577, 1569012590, 1569012655, 1569012662, 1569012673, 1569012693, 1569012715, 1569012729, 1569012757, 1569012771, 1569012788, 1569012818, 1569012839, 1569012896, 1569012898, 1569012901, 1569012922, 1569012948, 1569013045, 1569013058, 1569013063, 1569013125, 1569013130, 1569013156, 1569013169, 1569013185, 1569013186, 1569013193, 1569013198, 1569013247, 1569013292, 1569013366, 1569013419, 1569013462, 1569013531, 1569013594, 1569013627, 1569013632, 1569013684, 1569013708, 1569013710, 1569013752, 1569013769, 1569013871, 1569013929, 1569013940, 1569013960, 1569013962, 1569013979, 1569013994, 1569014029, 1569014042, 1569014055, 1569014156, 1569014161, 1569014197, 1569014273, 1569014327, 1569014333, 1569014358, 1569014375, 1569014385, 1569014399, 1569014414, 1569014423, 1569014455,

In [592]:
N = 20672783502493917028427
# solve_prime(N, 10000)

found potential roots: 1230 [143780330752, 143780330890, 143780330918, 143780330986, 143780330992, 143780331263, 143780331522, 143780331938, 143780332059, 143780332103, 143780332215, 143780332245, 143780332285, 143780332500, 143780332552, 143780332647, 143780332773, 143780333608, 143780333741, 143780334359, 143780334557, 143780335055, 143780335729, 143780335748, 143780336159, 143780337204, 143780337652, 143780337775, 143780338333, 143780338405, 143780339427, 143780340003, 143780340043, 143780341777, 143780341946, 143780342249, 143780342268, 143780342589, 143780342721, 143780342787, 143780343336, 143780343351, 143780344290, 143780344786, 143780345022, 143780345434, 143780345904, 143780347090, 143780347345, 143780348605, 143780348994, 143780349221, 143780350047, 143780350991, 143780351008, 143780351329, 143780351625, 143780352275, 143780352781, 143780353195, 143780353474, 143780353707, 143780354298, 143780354659, 143780355023, 143780357191, 143780357274, 143780358120, 143780358166, 14378

In [613]:
# Problem 7.9.3
def is_echelon(rowlist, k=0):
    """
    k: The first index which row could contain non-zero element in echelon form.
    """
    if not rowlist:
        return True  # zero matrix is echelon
    row, rest = rowlist[0], rowlist[1:]

    if any([row[i] != 0 for i in range(min(k, len(row)))]):
        return False
    
    # update k to the next valid position
    if k < len(row):
        k = next(i for i in range(k, len(row)) if row[i] != 0)
        k += 1

    return is_echelon(rest, k)

[
    is_echelon([[]]),
    is_echelon([[1]]),
    is_echelon([[1, 2, 3]]),
    is_echelon([[2, 1, 0], [0, -4, 0], [0, 0, 1]]),
    is_echelon([[2, 1, 0], [-4, 0, 0], [0, 0, 1]]),
    is_echelon([[2, 1, 0], [0, 3, 0], [1, 0, 1]]),
    is_echelon([[1, 1, 1, 1, 1], [0, 2, 0, 1, 3], [0, 0, 0, 5, 3]]),
]

[True, True, True, True, False, False, True]

In [670]:
# Problem 7.9.6
# TODO: why could the columns in the non leading zero position of each row be ignored?
from book.vecutil import zero_vec

def echelon_solve(row_list, label_list, b):
    x = zero_vec(row_list[0].D)
    for row, bi in reversed(list(zip(row_list, b))):
        for c in label_list:
             if row[c] != 0: 
                x[c] = (bi - x*row)/row[c]
                break
    return x

def test_echelon_solve(rows, b):
    label_list = list(range(len(rows[0])))
    row_list = [list2vec(r) for r in rows]
    solution = echelon_solve(row_list, label_list, b)

    if rowdict2mat(row_list) * solution == list2vec(b):
        return solution

    return None

[
    # problem 7.9.4
    test_echelon_solve([
        [10, 2, -3, 53],
        [0, 0, 1, 2013]
    ], [1, 3]),
    test_echelon_solve([
        [2, 0, 1, 3],
        [0, 0, 5, 3],
        [0, 0, 0, 1]
    ], [1, -1, 3]),
    test_echelon_solve([
        [2, 2, 4, 3, 2],
        [0, 0, -1, 11, 1],
        [0, 0, 0, 0, 5]
    ], [2, 0, 10]),

    # problem 7.9.5

    # (a), this one doesn't have a solution
    test_echelon_solve([ 
        [1, 3, -2, 1, 0],
        [0, 0, 2, -3, 0],
        [0, 0, 0, 0, 0]
    ], [5, 3, 2]),

    test_echelon_solve([ 
        [1, 2, -8, -4, 0],
        [0, 0, 2, 12, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0]
    ], [5, 4, 0, 0]),

    # problem 7.9.6
    test_echelon_solve([ 
        [one, 0, one, one, 0],
        [0, one, 0, 0, one],
        [0, 0, one, 0, one],
        [0, 0, 0, 0, one]
    ], [one, 0, one, one]),

    test_echelon_solve([ 
        [one, one, 0, one, 0],
        [0, one, 0, one, one],
        [0, 0, one, 0, one],
        [0, 0, 0, 0, 0]
    ], [one, 0, one, 0]),
]

[Vec({0, 1, 2, 3},{2: 3.0, 0: 1.0}),
 Vec({0, 1, 2, 3},{3: 3.0, 2: -2.0, 0: -3.0}),
 Vec({0, 1, 2, 3, 4},{4: 2.0, 2: 2.0, 0: -5.0}),
 None,
 Vec({0, 1, 2, 3, 4},{2: 2.0, 0: 21.0}),
 Vec({0, 1, 2, 3, 4},{4: one, 2: 0, 1: one, 0: one}),
 Vec({0, 1, 2, 3, 4},{2: one, 1: 0, 0: one})]

In [ ]:
# Our own version of solve
def solve(A, b):
    M = echelon.transformation(A)
    U = M*A
    col_label_list = sorted(A.D[1])
    U_rows_dict = mat2rowdict(U)
    row_list = [U_rows_dict[i] for i in sorted(U_rows_dict)]
    return echelon_solve(row_list,col_label_list, M*b)
    